In [1]:
# Imports
import pandas as pd
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel, VoigtModel, LinearModel, ConstantModel, SkewedGaussianModel
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from statistics import mean, median
from scipy.stats import skewnorm, gamma
from tqdm import tqdm
import random
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import numpy as np
#from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from statistics import mean, median
from tensorflow.keras import models, layers
from tensorflow.keras.layers import BatchNormalization, Activation, Conv1D, Input, Flatten, Dense, Dropout, MaxPool1D
from tensorflow.keras.optimizers import schedules, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import HeNormal
import tensorflow as tf
import keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
from keras import regularizers, losses, metrics

In [2]:
# Load the dataset
df = pd.read_csv('train_dataset_RAW.csv')

In [3]:
# Parse the dataset
c = df['C']
si = df['Si']
mn = df['Mn']
cr = df['Cr']
mo = df['Mo']
ni = df['Ni']
cu = df['Cu']
fe = df['Fe']
clean_df=df.T
clean_df.drop(clean_df.tail(8).index, inplace=True)
clean_df.index = clean_df.index.to_series().astype(str).str.replace('X','',regex=True).astype(float)
clean_df.index = clean_df.index.astype(float)
clean_df

,0,1,2,3,4,5,6,7,8,9,...,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099
200.000000,9.733174,2.764870,14.982197,3.433072,-14.350874,13.590914,-32.173777,-0.337839,0.095139,4.911503,...,34.356770,1.541407,-11.188400,-4.764745,17.755420,11.464980,7.214492,17.486760,18.985150,31.567410
200.020004,8.501220,5.715363,12.985340,40.284933,33.932075,27.381358,16.010137,20.696469,16.823974,-3.830110,...,26.562210,24.921380,5.188338,17.828350,15.011920,2.650868,2.815397,23.571810,-5.931100,47.484850
200.039993,8.185836,9.075145,-9.131695,20.909809,30.283853,10.892613,19.995555,6.131658,14.222372,8.939063,...,8.411401,26.746360,2.822810,-7.720987,13.857220,40.141830,-5.028434,41.003110,12.663690,15.980180
200.059998,10.380965,23.995839,14.757876,10.928583,14.815833,10.949070,4.629330,5.899498,25.952573,22.124660,...,9.687712,18.177650,8.133810,12.964820,22.038470,9.387419,21.517600,7.215416,35.650900,12.407760
200.080002,12.580477,9.274707,16.636502,25.688599,6.622596,6.387584,-2.785522,6.883508,16.266788,-6.250090,...,17.365740,6.131555,15.975730,28.072050,25.550930,12.663700,24.369180,28.663510,-8.056412,32.913890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999.940002,27.380685,26.018726,29.268105,33.352259,26.128323,33.117194,34.303266,18.549349,33.652233,19.506089,...,15.629830,29.551440,18.083940,40.349440,30.997920,23.051060,27.396950,29.035530,26.882880,26.614880
999.960022,35.267377,10.345581,24.442102,24.951109,25.739524,32.080784,47.102677,14.325021,30.861567,22.880584,...,14.686510,22.531140,19.195470,36.889110,33.799220,27.086520,20.645870,26.611240,31.810050,26.387350
999.979980,35.267366,10.345571,24.442092,24.951098,25.739513,32.080774,47.102667,14.325010,30.861557,22.880573,...,14.686500,22.531130,19.195460,36.889100,33.799210,27.086510,20.645860,26.611230,31.810040,26.387340
1000.000000,29.542232,16.072993,22.580933,26.604689,22.445700,33.588172,37.856679,24.928982,27.197287,22.191502,...,16.675570,16.812950,22.122050,33.082060,30.667850,29.890450,22.671110,29.645090,28.836510,30.120500


In [4]:
# Divide data into training/validation and testing set and reshape
unique_test_id = [1, 6, 14, 24, 26, 30, 39]
test_id = [list(range(i*50,i*50+50)) for i in unique_test_id]
test_id = [item for sublist in test_id for item in sublist]
x_train_val = clean_df.drop(test_id, axis=1).to_numpy()
y_train_val = cr.drop(test_id).to_numpy()
x_test = clean_df.iloc[:,test_id].to_numpy()
y_test = cr.iloc[test_id].to_numpy()
x_train_val = np.swapaxes(x_train_val, 0, 1)
x_test = np.swapaxes(x_test, 0, 1)

In [7]:
# Scale data

x_scaler_mean = np.mean(x_train_val)
x_scaler_std = np.std(x_train_val)

X_train_val = (x_train_val[:,:, np.newaxis]-x_scaler_mean)/x_scaler_std
X_test = (x_test[:, :, np.newaxis]-x_scaler_mean)/x_scaler_std

In [13]:
# Model architecture
def buildmodel():
    model = models.Sequential()
    model.add(layers.Conv1D(3, 3, activation='relu', input_shape = (40002 ,1)))
    model.add(layers.MaxPooling1D(pool_size = 2))
    model.add(layers.Conv1D(5, 5, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size = 2))
    model.add(layers.Conv1D(7, 7, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="linear"))



    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0025), metrics=['mae'])
    return model
    
model = buildmodel()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 40000, 3)          12        
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 20000, 3)         0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 19996, 5)          80        
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 9998, 5)          0         
 1D)                                                             
                                                                 
 conv1d_8 (Conv1D)           (None, 9992, 7)           252       
                                                                 
 flatten_2 (Flatten)         (None, 69944)            

In [ ]:
#XVD
k_selections = 50
test_sample_size = 5
RMSE_list = np.zeros(k_selections)
for k in tqdm(range(k_selections)):
    # Divide training/validation data into separte datasets and reshape
    unique_val_id = random.sample(range(35),test_sample_size)
    val_id = [list(range(i*50,i*50+50)) for i in unique_val_id]
    val_id = [item for sublist in val_id for item in sublist]
    x_val = X_train_val[val_id]
    y_val = y_train_val[val_id]
    x_train = np.delete(X_train_val, val_id, axis=0)
    y_train = np.delete(y_train_val, val_id)
    #X_train = x_train[:,:, np.newaxis]
    #X_val = x_val[:,:, np.newaxis]
    # Fit model
    history = model.fit(x_train, y_train, epochs=100, batch_size=1)
    # Have model make prediction
    y_pred = model.predict(x_val)
    # Calculate and append RMSE
    RMSE_list[k] = np.sqrt(np.mean(y_pred-y_val)**2)  
    # Reset weights
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))
    
plt.hist(RMSE_list)


  0%|                                                                                                                          | 0/50 [00:00<?, ?it/s]

Epoch 1/100
1500/1500 [==============================] - 24s 16ms/step - loss: 42.2754 - mae: 4.3907
Epoch 2/100
1500/1500 [==============================] - 24s 16ms/step - loss: 30.9898 - mae: 3.6823
Epoch 3/100
1500/1500 [==============================] - 23s 16ms/step - loss: 19.4246 - mae: 3.0679
Epoch 4/100
1500/1500 [==============================] - 27s 18ms/step - loss: 9.5890 - mae: 2.3598
Epoch 5/100
1500/1500 [==============================] - 25s 17ms/step - loss: 5.8122 - mae: 1.8485
Epoch 6/100
1500/1500 [==============================] - 23s 15ms/step - loss: 4.7977 - mae: 1.6816
Epoch 7/100
1500/1500 [==============================] - 23s 15ms/step - loss: 4.9318 - mae: 1.6627
Epoch 8/100
1500/1500 [==============================] - 22s 15ms/step - loss: 3.9898 - mae: 1.5250
Epoch 9/100
1500/1500 [==============================] - 23s 16ms/step - loss: 3.9774 - mae: 1.5342
Epoch 10/100
1500/1500 [==============================] - 26s 17ms/step - loss: 4.3150 - mae: 1.5

1500/1500 [==============================] - 25s 17ms/step - loss: 1.0135 - mae: 0.7595
Epoch 83/100
1500/1500 [==============================] - 26s 17ms/step - loss: 1.0148 - mae: 0.7655
Epoch 84/100
1500/1500 [==============================] - 22s 15ms/step - loss: 0.9678 - mae: 0.7645
Epoch 85/100
1500/1500 [==============================] - 23s 15ms/step - loss: 1.2070 - mae: 0.8248
Epoch 86/100
1500/1500 [==============================] - 23s 15ms/step - loss: 1.0770 - mae: 0.7826
Epoch 87/100
1500/1500 [==============================] - 24s 16ms/step - loss: 1.0772 - mae: 0.7869
Epoch 88/100
1500/1500 [==============================] - 23s 15ms/step - loss: 0.8620 - mae: 0.7220
Epoch 89/100
1500/1500 [==============================] - 24s 16ms/step - loss: 1.0431 - mae: 0.7625
Epoch 90/100
1500/1500 [==============================] - 23s 15ms/step - loss: 1.1393 - mae: 0.8191
Epoch 91/100
1500/1500 [==============================] - 23s 15ms/step - loss: 1.1023 - mae: 0.7927
Epo

  2%|██▏                                                                                                          | 1/50 [40:01<32:41:03, 2401.29s/it]

Epoch 1/100
1500/1500 [==============================] - 21s 14ms/step - loss: 40.2380 - mae: 4.1390
Epoch 2/100
1500/1500 [==============================] - 22s 15ms/step - loss: 29.8828 - mae: 3.5640
Epoch 3/100
1500/1500 [==============================] - 22s 15ms/step - loss: 24.6538 - mae: 3.2987
Epoch 4/100
1500/1500 [==============================] - 22s 14ms/step - loss: 11.6695 - mae: 2.4580
Epoch 5/100
1500/1500 [==============================] - 23s 15ms/step - loss: 5.1752 - mae: 1.7390
Epoch 6/100
1500/1500 [==============================] - 22s 15ms/step - loss: 3.4233 - mae: 1.4226
Epoch 7/100
1500/1500 [==============================] - 22s 15ms/step - loss: 3.0286 - mae: 1.3326
Epoch 8/100
1500/1500 [==============================] - 22s 15ms/step - loss: 2.8661 - mae: 1.2857
Epoch 9/100
1500/1500 [==============================] - 22s 15ms/step - loss: 2.9933 - mae: 1.3400
Epoch 10/100
1500/1500 [==============================] - 22s 15ms/step - loss: 2.6737 - mae: 1.

...

 44%|██████████████████████████████████████████████▏                                                          | 22/50 [30:07:01<29:20:29, 3772.50s/it]

Epoch 1/100
1500/1500 [==============================] - 48s 32ms/step - loss: 36.0425 - mae: 3.9150
Epoch 2/100
1500/1500 [==============================] - 49s 32ms/step - loss: 26.6235 - mae: 3.2407
Epoch 3/100
1500/1500 [==============================] - 47s 31ms/step - loss: 20.4210 - mae: 2.9241
Epoch 4/100
1500/1500 [==============================] - 47s 31ms/step - loss: 10.5113 - mae: 2.3158
Epoch 5/100
1500/1500 [==============================] - 48s 32ms/step - loss: 5.3124 - mae: 1.7328
Epoch 6/100
1500/1500 [==============================] - 47s 32ms/step - loss: 3.3953 - mae: 1.4183
Epoch 7/100
1500/1500 [==============================] - 49s 32ms/step - loss: 2.7278 - mae: 1.2831
Epoch 8/100
1500/1500 [==============================] - 48s 32ms/step - loss: 3.0149 - mae: 1.3241
Epoch 9/100
1500/1500 [==============================] - 47s 32ms/step - loss: 2.9660 - mae: 1.3099
Epoch 10/100
1500/1500 [==============================] - 47s 31ms/step - loss: 2.8555 - mae: 1.

1500/1500 [==============================] - 50s 33ms/step - loss: 0.7872 - mae: 0.6678
Epoch 83/100
1500/1500 [==============================] - 49s 32ms/step - loss: 1.0964 - mae: 0.7925
Epoch 84/100
1500/1500 [==============================] - 53s 36ms/step - loss: 0.9083 - mae: 0.7219
Epoch 85/100
1500/1500 [==============================] - 48s 32ms/step - loss: 0.8625 - mae: 0.6996
Epoch 86/100
1500/1500 [==============================] - 48s 32ms/step - loss: 0.9749 - mae: 0.7273
Epoch 87/100
1500/1500 [==============================] - 48s 32ms/step - loss: 0.8637 - mae: 0.6959
Epoch 88/100
1500/1500 [==============================] - 48s 32ms/step - loss: 0.8999 - mae: 0.7159
Epoch 89/100
1500/1500 [==============================] - 49s 33ms/step - loss: 1.0549 - mae: 0.7436
Epoch 90/100
1500/1500 [==============================] - 48s 32ms/step - loss: 0.9181 - mae: 0.7169
Epoch 91/100
1500/1500 [==============================] - 48s 32ms/step - loss: 0.9314 - mae: 0.7194
Epo

 46%|████████████████████████████████████████████████▎                                                        | 23/50 [31:27:12<30:37:54, 4084.22s/it]

Epoch 1/100
1500/1500 [==============================] - 47s 32ms/step - loss: 45.3302 - mae: 4.1879
Epoch 2/100
1500/1500 [==============================] - 49s 33ms/step - loss: 28.3874 - mae: 3.2656
Epoch 3/100
1500/1500 [==============================] - 47s 32ms/step - loss: 26.7917 - mae: 3.1684
Epoch 4/100
1500/1500 [==============================] - 49s 32ms/step - loss: 25.6556 - mae: 3.1390
Epoch 5/100
1500/1500 [==============================] - 50s 34ms/step - loss: 22.8486 - mae: 3.0397
Epoch 6/100
1500/1500 [==============================] - 50s 33ms/step - loss: 18.7590 - mae: 2.8148
Epoch 7/100
1500/1500 [==============================] - 50s 33ms/step - loss: 11.2572 - mae: 2.4435
Epoch 8/100
1500/1500 [==============================] - 50s 34ms/step - loss: 5.9982 - mae: 1.8756
Epoch 9/100
1500/1500 [==============================] - 50s 33ms/step - loss: 3.4335 - mae: 1.4114
Epoch 10/100
1500/1500 [==============================] - 50s 33ms/step - loss: 2.8091 - mae:

In [ ]:
np.mean(RMSE_list)

In [ ]:
plt.plot(history.history['loss'][70:])

In [ ]:
#XVD mini-batch
k_selections = 50
test_sample_size = 5
RMSE_list2 = np.zeros(k_selections)
for k in tqdm(range(k_selections)):
    # Divide training/validation data into separte datasets and reshape
    unique_val_id = random.sample(range(35),test_sample_size)
    val_id = [list(range(i*50,i*50+50)) for i in unique_val_id]
    val_id = [item for sublist in val_id for item in sublist]
    x_val = x_train_val[val_id]
    y_val = y_train_val[val_id]
    x_train = np.delete(x_train_val, val_id, axis=0)
    y_train = np.delete(y_train_val, val_id)
    X_train = x_train[:,:, np.newaxis]
    X_val = x_val[:,:, np.newaxis]
    # Fit model
    history2 = model.fit(X_train, y_train, epochs=150, batch_size=4)
    # Have model make prediction
    y_pred = model.predict(X_val)
    # Calculate and append RMSE
    RMSE_list2[k] = np.sqrt(np.mean(y_pred-y_val)**2)  
    # Reset weights
    for l in model.layers:
        if hasattr(l,"kernel_initializer"):
            l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
        if hasattr(l,"bias_initializer"):
            l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
        if hasattr(l,"recurrent_initializer"):
            l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))
    
plt.hist(RMSE_list2)

In [ ]:
for l in model.layers:
    if hasattr(l,"kernel_initializer"):
        l.kernel.assign(l.kernel_initializer(tf.shape(l.kernel)))
    if hasattr(l,"bias_initializer"):
        l.bias.assign(l.bias_initializer(tf.shape(l.bias)))
    if hasattr(l,"recurrent_initializer"):
        l.recurrent_kernel.assign(l.recurrent_initializer(tf.shape(l.recurrent_kernel)))